In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import h5py
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
%matplotlib inline
from resnet import *
import torch.optim as optim
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import time
import math
from utils import AverageMeter
import argparse
import utils

In [2]:
def parse_args():
    parser = argparse.ArgumentParser(description='alzheimer desease recognnition')
    parser.add_argument('--train_images', default='/data/beast/df/alzheimer/train.h5')
    parser.add_argument('--train_labels', default='/data/beast/df/alzheimer/train.csv')
    parser.add_argument('--val_images', default='/data/beast/df/alzheimer/val.h5')
    parser.add_argument('--val_labels', default='/data/beast/df/alzheimer/val.csv')
    parser.add_argument('--out_dir', default='/data/beast/df/alzheimer/out_model')
    parser.add_argument('--phase', default='train', choices=['train', 'val', 'test', 'predict'])
    parser.add_argument('--model', default='resnet34')
    parser.add_argument('--weight', default=None)
    parser.add_argument('--lr', default=1e-3, type=float)
    parser.add_argument('--mom', default=0.9, type=float)
    parser.add_argument('--wd', default=1e-4, type=float)
    parser.add_argument('--fix', default=50, type=int)
    parser.add_argument('--step', default=40, type=int)
    parser.add_argument('--epoch', default=120, type=int)
    parser.add_argument('--display', default=8, type=int)
    parser.add_argument('--num_workers', default=2, type=int)
    parser.add_argument('--batch_size', default=32, type=int)
    return parser.parse_args(args=[])

opt = parse_args()

In [3]:
def calc_stat_info(infile):
    f = h5py.File(infile)
    images = f['data']
    mean = np.mean(images)
    std = np.std(images)
    return mean, std

mean, std = calc_stat_info(opt.train_images)
print('mean:\t{}'.format(mean))
print('std:\t{}'.format(std))

class AlzheimerDS(Dataset):
    def __init__(self, infile, label_file):
        f = h5py.File(infile)
        images = f['data']
        self.images = np.transpose(images, [0,1,3,2,4])
#         self.mean = np.mean(images)
#         self.std = np.std(images)
        self.mean = mean
        self.std = std
        self.images = (self.images - self.mean)/self.std
        self.images = torch.from_numpy(self.images).float()
        df_labels = pd.read_csv(label_file)
        self.labels = list(df_labels['label'])
    def __getitem__(self, item):
        image = self.images[item]
        return image, self.labels[item]
    def __len__(self):
        return self.images.shape[0]

mean:	260.4920959472656
std:	273.56634521484375


In [4]:
# ds = AlzheimerDS('/data/beast/df/alzheimer/train_pre_data.h5', 
#                 '/data/beast/df/alzheimer/train_pre_label.csv')

In [5]:
# train_dataloader = DataLoader(ds, batch_size=4, shuffle=True, num_workers=2)

In [6]:
# model = resnet34(num_classes=3, shortcut_type=True, sample_size=79, sample_duration=95)

In [7]:
def initial_cls_weights(cls):
    for m in cls.modules():
        if isinstance(m, nn.Conv2d):
            n = m.kernel_size[0]*m.kernel_size[1]*m.out_channels
            m.weight.data.normal_(0, math.sqrt(2./n))
            if m.bias is not None:
                m.bias.data.zero_()
        if isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, 0.01)
            m.bias.data.zero_()
        if isinstance(m, nn.Conv3d):
            n = m.kernel_size[0]*m.kernel_size[1]*m.kernel_size[2]*m.out_channels
            m.weight.data.normal_(0, math.sqrt(2./n))
            if m.bias is not None:
                m.bias.data.zero_()

In [8]:
# initial_cls_weights(model)

In [9]:
# model_cuda = nn.DataParallel(model).cuda()
# criterion = nn.CrossEntropyLoss().cuda()

In [10]:
# lr = 1e-3
# mom = 0.9
# wd = 1e-4

# optimizer = optim.SGD([
#     {'params': model.parameters()}
# ], lr=lr, momentum=mom, weight_decay=wd, nesterov=True)

In [11]:
def train(train_data_loader, model, criterion, optimizer, epoch, display):
    model.train()
    tot_pred = np.array([], dtype=int)
    tot_label = np.array([], dtype=int)
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    accuracy = AverageMeter()
    end = time.time()
    logger = []
    for num_iter, (images, labels) in enumerate(train_data_loader):
        data_time.update(time.time()-end)
#         print(labels)
        output = model(Variable(images.cuda()))
        loss = criterion(output, Variable(labels.cuda()))
        _,pred = torch.max(output, 1)
#         kap = np.array(1-utils.quadratic_weighted_kappa(pred.cpu().numpy(), labels.cpu().numpy()), dtype=np.float32)
#         loss1 = torch.autograd.Variable(torch.from_numpy(kap).cuda(), requires_grad=True)
#         loss = loss + loss1
#         print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_time.update(time.time()-end)
        pred = pred.cpu().data.numpy().squeeze()
        labels = labels.numpy().squeeze()
        tot_pred = np.append(tot_pred, pred)
        tot_label = np.append(tot_label, labels)
        losses.update(loss.data.cpu().numpy(), len(images))
        accuracy.update(np.equal(pred, labels).sum()/len(labels), len(labels))
        end = time.time()
        if (num_iter+1) % display == 0:
            correct = np.equal(tot_pred, tot_label).sum()/len(tot_pred)
            print_info = 'Epoch: [{0}][{1}/{2}]\tTime {batch_time.val:3f} ({batch_time.avg:.3f})\t'\
                'Data {data_time.avg:.3f}\t''Loss {loss.avg:.4f}\tAccuray {accuracy.avg:.4f}'.format(
                epoch, num_iter, len(train_data_loader),batch_time=batch_time, data_time=data_time,
                loss=losses, accuracy=accuracy
            )
            print(print_info)
            logger.append(print_info)
    kappa = utils.quadratic_weighted_kappa(tot_pred, tot_label)
#     print('kappa:\t{:.3f}'.format(kappa))
    return accuracy.avg, logger


def val(train_data_loader, model, criterion, optimizer, epoch, display):
    model.train()
    tot_pred = np.array([], dtype=int)
    tot_label = np.array([], dtype=int)
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    accuracy = AverageMeter()
    end = time.time()
    logger = []
    for num_iter, (images, labels) in enumerate(train_data_loader):
        data_time.update(time.time()-end)
#         print(labels)
        output = model(Variable(images.cuda()))
        loss = criterion(output, Variable(labels.cuda()))
#         print(loss)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        batch_time.update(time.time()-end)
        _,pred = torch.max(output, 1)
        pred = pred.cpu().data.numpy().squeeze()
        labels = labels.numpy().squeeze()
        tot_pred = np.append(tot_pred, pred)
        tot_label = np.append(tot_label, labels)
        losses.update(loss.data.cpu().numpy(), len(images))
        accuracy.update(np.equal(pred, labels).sum()/len(labels), len(labels))
        end = time.time()
        if (num_iter+1) % display == 0:
            correct = np.equal(tot_pred, tot_label).sum()/len(tot_pred)
            print_info = 'Epoch: [{0}][{1}/{2}]\tTime {batch_time.val:3f} ({batch_time.avg:.3f})\t'\
                'Data {data_time.avg:.3f}\t''Loss {loss.avg:.4f}\tAccuray {accuracy.avg:.4f}'.format(
                epoch, num_iter, len(train_data_loader),batch_time=batch_time, data_time=data_time,
                loss=losses, accuracy=accuracy
            )
            print(print_info)
            logger.append(print_info)
    kappa = utils.quadratic_weighted_kappa(tot_pred, tot_label)
    acc = np.sum(tot_pred == tot_label)/len(tot_pred)
    print(tot_pred)
    print(tot_label)
    print('kappa:\t{:.3f}, accuracy:\t{:.3f}'.format(kappa, acc))
    return accuracy.avg, logger

In [12]:
# for i in range(10):
#     train(train_dataloader, nn.DataParallel(model).cuda(), criterion, optimizer, i, 70)

In [13]:
def main(i=0):
    print('====> parse options')
    opt = parse_args()
    print(opt)
    
    os.makedirs(opt.out_dir, exist_ok=True)
#     time_stamp = time.strftime('%Y%m%d%H%M%S', time.localtime(time.time()))
    time_stamp = 'xxx_{}'.format(i)
    out_dir = os.path.join(opt.out_dir, 'alzheimer_recognition_{}'.format(time_stamp))
    os.makedirs(out_dir, exist_ok=True)
    print('====> building model:')
    model = resnet18(num_classes=3, shortcut_type=True, sample_size=79, sample_duration=95)
    initial_cls_weights(model)
    criterion = nn.CrossEntropyLoss(torch.from_numpy(np.array([2.0,1.0,2.0], dtype=np.float32))).cuda()
    if opt.phase == 'train':
#         train_ds = AlzheimerDS(opt.train_images, opt.train_labels)
#         train_dataloader = DataLoader(train_ds, batch_size=opt.batch_size, shuffle=True, 
#                                       num_workers=opt.num_workers, pin_memory=True)
        
#         train_ds = AlzheimerDS(opt.train_images, opt.train_labels)
#         train_dataloader = DataLoader(train_ds, batch_size=opt.batch_size, shuffle=True, 
#                                       num_workers=opt.num_workers, pin_memory=True)
#         val_ds = AlzheimerDS(opt.val_images, opt.val_labels)
#         val_dataloader = DataLoader(val_ds, batch_size=opt.batch_size, shuffle=True, 
#                                       num_workers=opt.num_workers, pin_memory=True)
        train_ds = AlzheimerDS('/data/beast/df/alzheimer/train_{}.h5'.format(i), '/data/beast/df/alzheimer/train_{}.csv'.format(i))
        train_dataloader = DataLoader(train_ds, batch_size=opt.batch_size, shuffle=True, 
                                      num_workers=opt.num_workers, pin_memory=True)
        val_ds = AlzheimerDS('/data/beast/df/alzheimer/val_{}.h5'.format(i), '/data/beast/df/alzheimer/val_{}.csv'.format(i))
        val_dataloader = DataLoader(val_ds, batch_size=opt.batch_size, shuffle=True, 
                                      num_workers=opt.num_workers, pin_memory=True)
        beat_acc = 0.7
        for epoch in range(opt.epoch):
            if epoch < opt.fix:
                lr = opt.lr
            else:
                lr = opt.lr * (0.1 ** (epoch//opt.step))
            optimizer = None
            optimizer = optim.SGD([{'params': model.parameters()}], 
                                  lr=opt.lr, momentum=opt.mom, weight_decay=opt.wd, nesterov=True)
            
            _, _ = train(train_dataloader, nn.DataParallel(model).cuda(), criterion, optimizer, epoch, opt.display)
            acc, logger = val(val_dataloader, nn.DataParallel(model).cuda(), criterion, optimizer, epoch, opt.display)
            if acc > beat_acc:
                print('\ncurrent best accuracy is: {}\n'.format(acc))
                beat_acc = acc
                saved_model_name = os.path.join(out_dir, 'alzheimer_recognition_{:04d}_best.pth'.format(epoch))
                torch.save(model.cpu().state_dict(), saved_model_name)
                print('====> save model:\t{}'.format(saved_model_name))
                
    
                
                

In [ ]:
# main()
for i in range(5):
    main(i)

====> parse options
Namespace(batch_size=32, display=8, epoch=120, fix=50, lr=0.001, model='resnet34', mom=0.9, num_workers=2, out_dir='/data/beast/df/alzheimer/out_model', phase='train', step=40, train_images='/data/beast/df/alzheimer/train.h5', train_labels='/data/beast/df/alzheimer/train.csv', val_images='/data/beast/df/alzheimer/val.h5', val_labels='/data/beast/df/alzheimer/val.csv', wd=0.0001, weight=None)
====> building model:


/home/ubuntu/beast/code/DFSolution/alzheimer/resnet.py:145: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


Epoch: [0][7/9]	Time 0.161195 (0.231)	Data 0.049	Loss 1.0966	Accuray 0.4180
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 0 2 1 1 1 1 2 2 0 2 0 1 1 1 1 0 0 2 2 1 1 0 1 2 0 1 2]
kappa:	0.000, accuracy:	0.233
Epoch: [1][7/9]	Time 0.168042 (0.205)	Data 0.038	Loss 1.0714	Accuray 0.3828
[2 1 2 1 2 1 2 2 1 1 2 1 1 2 1 2 2 1 2 1 1 1 2 2 1 1 1 2 1 1]
[2 1 2 1 2 1 0 0 1 1 2 1 1 2 1 2 0 1 0 1 1 1 2 0 1 0 2 0 1 1]
kappa:	0.042, accuracy:	0.733

current best accuracy is: 0.7333333333333333

====> save model:	/data/beast/df/alzheimer/out_model/alzheimer_recognition_xxx_0/alzheimer_recognition_0001_best.pth
Epoch: [2][7/9]	Time 0.165627 (0.214)	Data 0.041	Loss 1.0311	Accuray 0.6328
[1 2 1 1 2 2 2 2 1 2 2 2 1 2 2 2 2 2 1 2 2 2 2 1 2 2 2 2 2 2]
[1 2 1 1 0 0 2 0 1 1 1 0 1 0 1 2 2 2 1 2 1 1 1 1 2 0 1 2 1 0]
kappa:	0.013, accuracy:	0.500
Epoch: [3][7/9]	Time 0.169170 (0.207)	Data 0.040	Loss 0.9960	Accuray 0.6953
[2 1 2 1 1 2 1 2 2 1 1 1 2 1 1 1 1 1 2 2 1 1 2 2 2 2 1 1 2 1]
[0 1 2 1

Epoch: [33][7/9]	Time 0.167897 (0.207)	Data 0.039	Loss 0.1670	Accuray 0.9922
[1 1 0 2 1 2 2 1 1 0 1 1 2 1 1 1 1 1 2 0 2 1 1 0 0 2 2 2 0 1]
[1 1 0 0 1 0 2 1 1 0 1 1 2 1 1 1 1 1 2 0 0 1 1 2 2 2 2 2 0 1]
kappa:	0.329, accuracy:	0.833
Epoch: [34][7/9]	Time 0.167029 (0.210)	Data 0.042	Loss 0.1769	Accuray 0.9805
[1 0 2 0 1 0 2 1 1 2 2 1 1 1 1 1 1 1 2 1 2 2 0 1 0 2 0 1 1 0]
[1 0 2 2 1 2 2 1 1 0 2 1 1 1 1 1 1 1 0 1 2 0 0 1 2 2 0 1 1 0]
kappa:	0.198, accuracy:	0.800
Epoch: [35][7/9]	Time 0.167249 (0.206)	Data 0.040	Loss 0.1404	Accuray 0.9883
[2 2 2 0 1 1 1 1 2 1 0 1 2 1 1 2 2 0 1 1 2 2 2 1 1 1 1 2 0 1]
[2 0 0 2 1 1 1 1 2 1 0 1 2 1 1 0 2 0 1 1 0 2 2 1 1 1 1 0 2 1]
kappa:	0.052, accuracy:	0.767
Epoch: [36][7/9]	Time 0.164521 (0.208)	Data 0.036	Loss 0.1470	Accuray 0.9805
[2 1 1 1 1 0 1 2 1 0 1 0 1 0 0 2 2 1 1 2 1 2 1 1 1 2 0 2 1 0]
[0 1 1 1 1 0 1 2 1 2 1 0 1 0 2 0 2 1 1 0 1 2 1 1 1 2 2 2 1 0]
kappa:	0.198, accuracy:	0.800
Epoch: [37][7/9]	Time 0.164501 (0.208)	Data 0.041	Loss 0.1613	Accuray 0.9688

Epoch: [68][7/9]	Time 0.166584 (0.209)	Data 0.040	Loss 0.0494	Accuray 0.9922
[0 0 2 1 2 1 1 2 0 1 1 1 2 2 1 1 1 1 1 2 2 2 0 1 1 0 2 1 0 1]
[2 2 0 1 2 1 1 2 0 1 1 1 2 2 1 1 1 1 1 2 0 2 0 1 1 0 0 1 0 1]
kappa:	0.329, accuracy:	0.833
Epoch: [69][7/9]	Time 0.167309 (0.204)	Data 0.035	Loss 0.0453	Accuray 0.9883
[1 2 1 2 2 0 1 0 0 1 0 2 1 1 1 2 1 1 1 1 2 1 1 1 2 1 0 0 2 0]
[1 2 1 2 2 2 1 0 2 1 2 0 1 1 1 0 1 1 1 1 2 1 1 1 0 1 0 0 2 0]
kappa:	0.198, accuracy:	0.800
Epoch: [70][7/9]	Time 0.166312 (0.213)	Data 0.042	Loss 0.0410	Accuray 0.9883
[2 0 1 2 0 1 0 2 1 1 1 1 0 1 2 2 1 2 1 1 2 2 1 1 2 1 1 1 0 0]
[2 0 1 2 0 1 2 2 1 1 1 1 0 1 0 0 1 2 1 1 2 2 1 1 0 1 1 1 2 0]
kappa:	0.329, accuracy:	0.833
Epoch: [71][7/9]	Time 0.167446 (0.211)	Data 0.043	Loss 0.0710	Accuray 0.9805
[1 0 2 1 2 1 2 2 2 0 1 2 0 1 2 0 1 1 0 0 1 1 1 0 1 1 1 1 1 2]
[1 0 2 1 2 1 0 0 2 0 1 2 0 1 0 2 1 1 2 0 1 1 1 2 1 1 1 1 1 2]
kappa:	0.198, accuracy:	0.800
Epoch: [72][7/9]	Time 0.164090 (0.204)	Data 0.035	Loss 0.0396	Accuray 0.9883

...

Epoch: [85][7/9]	Time 0.167962 (0.210)	Data 0.042	Loss 0.0974	Accuray 0.9805
[0 2 0 1 1 0 2 2 1 1 1 0 2 1 1 2 1 2 1 1 1 1 1 0 1 2 2 2 1 0]
[0 2 0 1 1 0 0 2 1 1 1 0 0 1 1 2 1 0 1 1 1 1 1 2 1 2 2 2 1 2]
kappa:	0.329, accuracy:	0.833
Epoch: [86][7/9]	Time 0.172949 (0.209)	Data 0.039	Loss 0.0500	Accuray 0.9805
[1 2 1 2 1 2 2 1 1 1 2 0 1 1 0 0 2 1 1 1 1 2 0 2 1 0 1 2 0 1]
[1 0 1 0 1 2 0 1 1 1 2 0 1 1 0 2 2 1 1 1 1 2 2 2 1 0 1 2 0 1]
kappa:	0.329, accuracy:	0.833
Epoch: [87][7/9]	Time 0.169591 (0.212)	Data 0.038	Loss 0.0308	Accuray 0.9961
[2 2 1 2 2 0 1 1 1 1 1 1 2 0 1 0 1 0 2 2 1 2 2 1 1 1 0 1 1 2]
[2 2 1 0 2 0 1 1 1 1 1 1 2 0 1 0 1 2 2 0 1 2 2 1 1 1 0 1 1 0]
kappa:	0.461, accuracy:	0.867
Epoch: [88][7/9]	Time 0.167301 (0.213)	Data 0.045	Loss 0.0334	Accuray 0.9883
[2 1 2 1 1 0 2 1 0 2 1 1 2 1 1 2 2 1 2 1 1 1 1 0 0 1 0 2 2 1]
[2 1 0 1 1 2 2 1 0 0 1 1 0 1 1 2 2 1 2 1 1 1 1 0 2 1 0 0 2 1]
kappa:	0.191, accuracy:	0.800
Epoch: [89][7/9]	Time 0.166632 (0.212)	Data 0.043	Loss 0.0436	Accuray 0.9922

Epoch: [0][7/9]	Time 0.164842 (0.211)	Data 0.041	Loss 1.1167	Accuray 0.3516
[2 2 2 2 2 2 2 1 2 1 1 2 2 2 2 2 1 2 1 2 2 2 2 1 2 2 2 1 2 2]
[1 0 0 2 2 2 0 1 1 1 1 0 1 1 1 1 1 0 1 0 1 0 2 1 1 2 2 1 2 2]
kappa:	0.013, accuracy:	0.500
Epoch: [1][7/9]	Time 0.169754 (0.212)	Data 0.041	Loss 1.0630	Accuray 0.3789
[1 1 1 2 1 1 1 2 1 1 2 1 2 1 1 0 1 1 1 1 2 1 2 1 1 2 1 2 1 1]
[1 0 1 0 1 1 2 0 1 1 0 1 2 2 1 2 1 1 2 1 2 1 2 1 1 0 1 2 0 0]
kappa:	-0.105, accuracy:	0.633
Epoch: [2][7/9]	Time 0.167748 (0.226)	Data 0.051	Loss 1.0356	Accuray 0.6211
[2 2 2 2 2 2 2 2 1 2 1 2 2 2 2 1 2 2 1 1 1 2 2 2 1 2 1 1 2 1]
[2 0 2 2 1 0 2 1 1 2 1 0 0 0 0 1 2 2 1 1 1 0 1 2 1 1 1 1 1 1]
kappa:	0.020, accuracy:	0.600
Epoch: [3][7/9]	Time 0.166766 (0.212)	Data 0.043	Loss 1.0101	Accuray 0.6914
[0 2 0 2 1 0 1 2 2 0 1 2 2 2 1 0 1 2 2 0 2 0 1 2 2 2 2 0 2 2]
[2 1 2 0 1 2 1 1 1 2 1 0 0 2 1 0 1 1 2 1 0 2 1 2 0 1 1 1 0 1]
kappa:	-0.378, accuracy:	0.333
Epoch: [4][7/9]	Time 0.166625 (0.206)	Data 0.034	Loss 0.9666	Accuray 0.7031
[1

Epoch: [35][7/9]	Time 0.167953 (0.206)	Data 0.038	Loss 0.1949	Accuray 0.9727
[1 1 1 1 1 0 0 1 1 0 1 0 0 1 0 0 1 1 2 0 1 1 2 1 1 0 0 1 1 2]
[1 1 1 1 1 2 2 1 0 0 1 2 2 1 2 0 1 1 2 0 1 1 2 0 1 2 0 1 1 0]
kappa:	-0.054, accuracy:	0.700
Epoch: [36][7/9]	Time 0.169964 (0.215)	Data 0.044	Loss 0.1602	Accuray 0.9805
[1 1 2 2 2 1 2 2 1 2 1 2 1 2 1 1 1 0 2 1 1 0 0 2 1 1 0 1 2 1]
[1 1 0 0 2 1 2 0 1 0 1 0 1 0 1 1 1 2 2 1 1 2 0 2 1 1 2 1 2 1]
kappa:	-0.219, accuracy:	0.700
Epoch: [37][7/9]	Time 0.167610 (0.205)	Data 0.035	Loss 0.1246	Accuray 0.9883
[1 1 1 1 2 1 0 1 2 2 0 2 1 2 1 2 0 0 2 1 1 1 2 2 1 1 1 1 2 2]
[1 1 1 1 0 1 0 1 0 0 2 2 1 2 1 0 2 2 0 1 1 1 2 2 1 1 1 1 0 2]
kappa:	-0.219, accuracy:	0.700
Epoch: [38][7/9]	Time 0.167840 (0.210)	Data 0.041	Loss 0.1245	Accuray 0.9766
[1 1 2 0 0 1 2 2 2 2 1 2 1 0 1 0 1 1 2 1 1 2 2 1 2 1 1 1 2 1]
[1 1 0 2 2 1 0 0 2 0 1 2 1 0 1 2 1 1 0 1 1 2 2 1 2 1 1 1 0 1]
kappa:	-0.219, accuracy:	0.700
Epoch: [39][7/9]	Time 0.164249 (0.211)	Data 0.040	Loss 0.0923	Accuray 0.

[2 2 0 0 1 2 2 2 1 0 0 0 1 0 2 1 1 1 1 1 1 1 1 0 1 1 2 2 1 1]
[0 2 2 0 1 2 2 0 1 0 0 2 1 2 2 1 1 1 1 1 1 1 1 0 1 1 0 2 1 1]
kappa:	0.198, accuracy:	0.800
Epoch: [70][7/9]	Time 0.170141 (0.214)	Data 0.045	Loss 0.0498	Accuray 0.9883
[0 1 0 0 1 1 1 0 2 2 1 1 1 2 1 2 1 1 2 1 0 1 2 1 1 2 0 0 0 1]
[0 1 0 0 1 1 1 0 0 0 1 1 1 2 1 0 1 1 2 1 2 1 2 1 1 2 2 2 2 1]
kappa:	0.069, accuracy:	0.767
Epoch: [71][7/9]	Time 0.167967 (0.212)	Data 0.043	Loss 0.0679	Accuray 0.9844
[2 1 1 2 2 0 1 1 1 0 1 1 1 0 1 1 0 2 0 1 1 2 1 0 1 2 2 1 1 2]
[0 1 1 2 2 2 1 1 1 0 1 0 1 2 1 1 2 0 0 1 1 2 1 0 1 2 2 1 1 0]
kappa:	0.134, accuracy:	0.767
Epoch: [72][7/9]	Time 0.165783 (0.205)	Data 0.035	Loss 0.0626	Accuray 0.9844
[2 1 1 0 0 2 1 1 2 1 1 0 1 2 2 2 1 0 1 1 2 1 1 0 0 1 0 2 1 1]
[2 1 1 0 2 2 1 1 2 1 1 2 1 2 0 0 1 0 1 1 2 1 1 2 0 1 0 0 1 1]
kappa:	0.198, accuracy:	0.800
Epoch: [73][7/9]	Time 0.168667 (0.221)	Data 0.050	Loss 0.0524	Accuray 0.9922
[1 0 0 1 2 1 0 1 2 2 2 2 2 1 1 1 2 1 2 0 1 1 0 1 1 1 0 1 2 1]
[1 0 0 1 0 1 0

[2 2 1 2 2 2 1 1 1 1 1 1 1 0 0 1 2 2 1 2 1 0 0 0 2 1 1 1 2 1]
[0 2 1 0 2 2 1 1 1 1 1 1 1 0 0 1 2 2 1 0 1 0 2 2 2 1 1 1 0 1]
kappa:	0.191, accuracy:	0.800
Epoch: [105][7/9]	Time 0.167713 (0.213)	Data 0.043	Loss 0.0732	Accuray 0.9805
[1 2 0 1 1 1 2 1 2 0 0 0 0 1 1 0 0 1 0 1 1 1 0 1 1 1 1 2 2 2]
[1 2 0 1 1 1 2 1 0 0 0 2 0 1 1 2 0 1 2 1 1 1 2 1 1 1 1 2 2 0]
kappa:	0.205, accuracy:	0.800
Epoch: [106][7/9]	Time 0.167312 (0.213)	Data 0.043	Loss 0.0608	Accuray 0.9844
[1 0 2 1 2 0 1 1 0 1 2 0 1 1 1 1 1 1 0 0 1 2 2 1 2 1 0 1 2 2]
[1 2 0 1 2 0 1 1 2 1 2 0 1 1 1 1 1 1 2 0 1 0 2 1 2 1 0 1 0 2]
kappa:	0.198, accuracy:	0.800
Epoch: [107][7/9]	Time 0.169176 (0.208)	Data 0.037	Loss 0.0513	Accuray 0.9883
[0 1 2 0 2 1 1 2 1 1 0 1 0 1 2 2 0 1 1 1 2 1 1 1 1 0 0 0 1 2]
[0 1 2 0 2 1 1 2 1 1 2 1 0 1 2 2 2 1 1 1 0 1 1 1 1 0 2 0 1 0]
kappa:	0.335, accuracy:	0.833
Epoch: [108][7/9]	Time 0.167517 (0.210)	Data 0.043	Loss 0.0420	Accuray 0.9883
[2 2 0 0 1 1 2 1 1 1 2 0 1 1 1 2 1 2 0 0 0 1 2 2 1 1 1 0 1 1]
[2 0 2 0 1

In [ ]:
# print('====> parse options')
# opt = parse_args()
# print(opt)
    
#     os.makedirs(opt.out_dir, exist_ok=True)
#     time_stamp = time.strftime('%Y%m%d%H%M%S', time.localtime(time.time()))
#     out_dir = os.path.join(opt.out_dir, 'alzheimer_recognition_{}'.format(time_stamp))
#     os.makedirs(out_dir, exist_ok=True)
#     print('====> building model:')
# model = resnet34(num_classes=3, shortcut_type=True, sample_size=79, sample_duration=95)
# initial_cls_weights(model)

In [ ]:
ls '/data/beast/df/alzheimer/out_model/alzheimer_recognition_20191018150905/'

In [ ]:
# model_weights = '/data/beast/df/alzheimer/out_model/alzheimer_recognition_20191022143605/alzheimer_recognition_0091_best.pth'
# model.load_state_dict(torch.load(model_weights))

In [ ]:
# def calc_stat_info(infile):
#     f = h5py.File(infile)
#     images = f['data']
#     mean = np.mean(images)
#     std = np.std(images)
#     return mean, std

# mean, std = calc_stat_info(opt.train_images)
# print('mean:\t{}'.format(mean))
# print('std:\t{}'.format(std))

In [ ]:
class AlzheimerPredictDS(Dataset):
    def __init__(self, infile, mean, std):
        f = h5py.File(infile)
        images = f['data']
        self.images = np.transpose(images, [0,1,3,2,4])
        self.mean = mean
        self.std = std
        self.images = (self.images - self.mean)/self.std
        self.images = torch.from_numpy(self.images).float()
    def __getitem__(self, item):
        image = self.images[item]
        return image, item
    def __len__(self):
        return self.images.shape[0]
    
# eval_ds = AlzheimerPredictDS('/data/beast/df/alzheimer/testa.h5', mean, std)
# eval_data_loader = DataLoader(eval_ds, batch_size=4, num_workers=2, shuffle=False, pin_memory=False)
# for i, (images, items) in enumerate(eval_data_loader):
#     print(images.shape)
#     print(items)

In [ ]:
def predict_eval(eval_data_loader, model, display):
    model.eval()
    tot_pred = np.array([], dtype=int)
    tot_items = np.array([], dtype=int)
    tot_pred_probs = np.array([], dtype=float)
    batch_time = AverageMeter()
    data_time = AverageMeter()
    end = time.time()
    logger = []
    for num_iter, (images, items) in enumerate(eval_data_loader):
        data_time.update(time.time()-end)
        output = model(Variable(images.cuda()))
#         print(output.cpu().data.numpy().squeeze())
        probs = torch.nn.functional.softmax(output.cpu())
        probs = probs.data.numpy().squeeze()
#         print(probs)
        tot_pred_probs = np.append(tot_pred_probs, probs)
        _,pred = torch.max(output, 1)
        pred = pred.cpu().data.numpy().squeeze()
        items = items.numpy().squeeze()
        batch_time.update(time.time()-end)
        tot_pred = np.append(tot_pred, pred)
        tot_items = np.append(tot_items, items)
        end = time.time()
    return tot_pred, tot_items, tot_pred_probs
        
# pred, items = predict_eval(eval_data_loader, nn.DataParallel(model).cuda(), 2)
# print(pred)
# print(items)

In [ ]:
def predict(infile, model_weights=None):
    print('====> parse options')
    opt = parse_args()
    print(opt)
    model = resnet18(num_classes=3, shortcut_type=True, sample_size=79, sample_duration=95)
    initial_cls_weights(model)
#     model_weights = '/data/beast/df/alzheimer/out_model/alzheimer_recognition_20191022165341/alzheimer_recognition_0056_best.pth'
#     if not model_weights:
    model.load_state_dict(torch.load(model_weights))
    eval_ds = AlzheimerPredictDS(infile, mean, std)
    eval_data_loader = DataLoader(eval_ds, batch_size=4, num_workers=2, shuffle=False, pin_memory=False)
    pred, items, probs = predict_eval(eval_data_loader, nn.DataParallel(model).cuda(), 2)
    return pred, items, probs

In [ ]:
infiles = []
# infiles.append('/data/beast/df/alzheimer/out_model/alzheimer_recognition_xxx_0/alzheimer_recognition_0059_best.pth')
# infiles.append('/data/beast/df/alzheimer/out_model/alzheimer_recognition_xxx_1/alzheimer_recognition_0053_best.pth')
# infiles.append('/data/beast/df/alzheimer/out_model/alzheimer_recognition_xxx_2/alzheimer_recognition_0043_best.pth')
infiles.append('/data/beast/df/alzheimer/out_model/alzheimer_recognition_xxx_3/alzheimer_recognition_0050_best.pth')
# infiles.append('/data/beast/df/alzheimer/out_model/alzheimer_recognition_xxx_4/alzheimer_recognition_0080_best.pth')

In [ ]:
infile_a = '/data/beast/df/alzheimer/testa.h5'
infile_b = '/data/beast/df/alzheimer/testb.h5'
test_id = None
probs = []
for i in range(len(infiles)):
    pred_a, items_a, probs_a = predict(infile_a, infiles[0])
    pred_b, items_b, probs_b = predict(infile_b, infiles[0])
    test_id = ['testa_{}'.format(i) for i in items_a] + ['testb_{}'.format(i) for i in items_b]
    test_id = np.array(test_id)
    probs_ab = np.array([], dtype=float)
    probs_ab = np.append(probs_ab, probs_a)
    probs_ab = np.append(probs_ab, probs_b)
    probs.append(probs_ab)

In [ ]:
probs_res = None
for i in range(len(probs)):
    if probs_res is None:
        probs_res = probs[0].copy()
        print(probs_res.shape)
        continue
    probs_res = probs_res + probs[i]
print(probs_res.shape)
probs_res = np.resize(probs_res, (232,3))
print(probs_res.shape)
label_res = np.argmax(probs_res, axis=1)
label = np.array(label_res, dtype=int)
print(label.shape)
print(probs_res.shape)

In [ ]:
# infile_a = '/data/beast/df/alzheimer/testa.h5'
# infile_b = '/data/beast/df/alzheimer/testb.h5'
# pred_a, items_a, probs_a = predict(infile_a, infiles[0])
# pred_b, items_b, probs_b = predict(infile_b, infiles[0])
# test_id = ['testa_{}'.format(i) for i in items_a] + ['testb_{}'.format(i) for i in items_b]
# test_id = np.array(test_id)
# label = np.array([], dtype=int)
# label = np.append(label, pred_a)
# label = np.append(label, pred_b)
# # print(label)
# # print(test_id)
# print(label.shape)
# print(test_id.shape)

In [ ]:
print(test_id.shape)
print(label.shape)

In [ ]:
d = {'testa_id': test_id, 'label': label}
df_result = pd.DataFrame(data=d)
df_result.to_csv('./result_mean.csv', index=False)

In [ ]:
ls /data/beast/df/alzheimer/out_model/alzheimer_recognition_20191022143605

In [ ]:
infiles = []
infiles.append('/data/beast/df/alzheimer/out_model/alzheimer_recognition_xxx_0/alzheimer_recognition_0046_best.pth')
infiles.append('/data/beast/df/alzheimer/out_model/alzheimer_recognition_xxx_1/alzheimer_recognition_0067_best.pth')
infiles.append('/data/beast/df/alzheimer/out_model/alzheimer_recognition_xxx_2/alzheimer_recognition_0076_best.pth')
infiles.append('/data/beast/df/alzheimer/out_model/alzheimer_recognition_xxx_3/alzheimer_recognition_0057_best.pth')
infiles.append('/data/beast/df/alzheimer/out_model/alzheimer_recognition_xxx_4/alzheimer_recognition_0084_best.pth')